In [3]:
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as pl
import time
import numpy as np
from flask import Flask, render_template, request

app = Flask(__name__)
net= cv.dnn.readNetFromTensorflow("graph_opt.pb")
inWidth=368
inHeight=368
thr=0.2
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]
def pose_estimation(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]
    
    assert(len(BODY_PARTS) == out.shape[1])
    
    points = []
    
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponding body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]

        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
        
    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    return points

def body_part_length_estimation(height, estimated_image):
    # Your existing logic for body part length estimation
    # based on the user's height and camera frame goes here
    # This function should return a Pandas DataFrame containing the measurements
    # Example: measurements_df = ... (your DataFrame with measurements)
    z=np.sqrt(np.square(estimated_image[0][0]-estimated_image[13][0])+np.square(estimated_image[0][1]-estimated_image[13][1]))
    caliber=height/z
    chestlength=np.sqrt(np.square(estimated_image[2][0]-estimated_image[5][0])+np.square(estimated_image[2][1]-estimated_image[5][1]))
    shoulder=chestlength*caliber*1.2
    chestlength=chestlength*caliber*2.5
    waist=np.sqrt(np.square(estimated_image[8][0]-estimated_image[11][0])+np.square(estimated_image[8][1]-estimated_image[11][1]))
    waist=waist*caliber*3.4
    wrist=waist/4
    frontlength=np.sqrt(np.square(estimated_image[1][0]-estimated_image[11][0])+np.square(estimated_image[1][1]-estimated_image[11][1]))
    frontlength=frontlength*caliber
    backlength=frontlength
    sleeves=np.sqrt(np.square(estimated_image[5][0]-estimated_image[7][0])+np.square(estimated_image[5][1]-estimated_image[7][1]))
    sleeves=sleeves*caliber
    pantlength=np.sqrt(np.square(estimated_image[11][0]-estimated_image[13][0])+np.square(estimated_image[11][1]-estimated_image[13][1]))
    pantlength=pantlength*caliber
    dict={"Type":["chestlength","shoulder","waist","wrist","frontlength","backlength","sleeves","pantlength"],
          "measurement(inch)":[chestlength,shoulder,waist,wrist,frontlength,backlength,sleeves,pantlength]}
    measurements_df= pd.DataFrame.from_dict(dict)
    return measurements_df

@app.route('/', methods=['GET', 'POST'])
def height_and_measurement():
    if request.method == 'POST':
        # Get the user's height from the form
        height = float(request.form['height'])
        TIMER = int(5)
        # Open the camera
        print("stand in A pose")
        cap = cv.VideoCapture(0)
        while True:

            # Read and display each frame
            ret, img = cap.read()
            cv.imshow('a', img)

            # check for the key pressed
            k = cv.waitKey(125)

            # set the key for the countdown
            # to begin. Here we set q
            # if key pressed is q
            if k == ord('q'):
                prev = time.time()

                while TIMER >= 0:
                    ret, img = cap.read()

                    # Display countdown on each frame
                    # specify the font and draw the
                    # countdown using puttext
                    font = cv.FONT_HERSHEY_SIMPLEX
                    cv.putText(img, str(TIMER),
                                (200, 250), font,
                                7, (0, 255, 255),
                                4, cv.LINE_AA)
                    cv.imshow('a', img)
                    cv.waitKey(125)

                    # current time
                    cur = time.time()

                    # Update and keep track of Countdown
                    # if time elapsed is one second
                    # then decrease the counter
                    if cur-prev >= 1:
                        prev = cur
                        TIMER = TIMER-1

                else:
                    ret, img = cap.read()

                    # Display the clicked frame for 2
                    # sec.You can increase time in
                    # waitKey also
                    cv.imshow('a', img)

                    # time for which image displayed
                    cv.waitKey(2000)

                    # Save the frame
                    cv.imwrite('new5.jpg', img)

                    # HERE we can reset the Countdown timer
                    # if we want more Capture without closing
                    # the camera

            # Press Esc to exit
            elif k == 27:
                break

        # close the camera
        cap.release()
        # close all the opened windows
        cv.destroyAllWindows()

        print("stand in side wise pose")

        TIMER = int(5)

        cap = cv.VideoCapture(0)
        while True:

            # Read and display each frame
            ret, img = cap.read()
            cv.imshow('a', img)

            # check for the key pressed
            k = cv.waitKey(125)

            # set the key for the countdown
            # to begin. Here we set q
            # if key pressed is q
            if k == ord('q'):
                prev = time.time()

                while TIMER >= 0:
                    ret, img = cap.read()

                    # Display countdown on each frame
                    # specify the font and draw the
                    # countdown using puttext
                    font = cv.FONT_HERSHEY_SIMPLEX
                    cv.putText(img, str(TIMER),
                                (200, 250), font,
                                7, (0, 255, 255),
                                4, cv.LINE_AA)
                    cv.imshow('a', img)
                    cv.waitKey(125)

                    # current time
                    cur = time.time()

                    # Update and keep track of Countdown
                    # if time elapsed is one second
                    # then decrease the counter
                    if cur-prev >= 1:
                        prev = cur
                        TIMER = TIMER-1

                else:
                    ret, img = cap.read()

                    # Display the clicked frame for 2
                    # sec.You can increase time in
                    # waitKey also
                    cv.imshow('a', img)

                    # time for which image displayed
                    cv.waitKey(2000)

                    # Save the frame
                    cv.imwrite('abcd.jpg', img)

                    # HERE we can reset the Countdown timer
                    # if we want more Capture without closing
                    # the camera

            # Press Esc to exit
            elif k == 27:
                break

        # close the camera
        cap.release()
        # close all the opened windows
        cv.destroyAllWindows()
        img =cv.imread("new5.jpg")
        estimated_image=pose_estimation(img)

        # Process the camera footage here to get the frame
        # Example: camera_frame = ... (your camera frame)

        # Call your existing function to estimate body part lengths
        measurements_df = body_part_length_estimation(height, estimated_image)

        # Optionally, you can display the DataFrame on the webpage
        return render_template('result.html', table=measurements_df.to_html(classes='table table-striped'))

    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Aug/2023 07:27:01] "GET / HTTP/1.1" 200 -


stand in A pose
stand in side wise pose


127.0.0.1 - - [07/Aug/2023 07:27:56] "POST / HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\sudhanshu raj\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2213, in __call__
    return self.wsgi_app(environ, start_response)
  File "C:\Users\sudhanshu raj\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2193, in wsgi_app
    response = self.handle_exception(e)
  File "C:\Users\sudhanshu raj\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\sudhanshu raj\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\sudhanshu raj\AppData\Local\Programs\Python\Python310\lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\sudhanshu raj\AppDat